<a href="https://colab.research.google.com/github/leomercanti/SP500_Momentum_Strategy_Backtesting/blob/main/SP500_Momentum_Strategy_Backtesting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **S&P500 Momentum Strategy and Backtesting Project**

In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt

### **Step 1: Data Acquisition**

Use the yfinance library to fetch historical price data for the S&P 500 index from January 2010 to September 2024.

In [ ]:
start_date = '2010-01-01'
end_date = '2024-09-30'
symbol = '^GSPC'  # S&P 500 Index

data = yf.download(symbol, start=start_date, end=end_date)
data['Date'] = data.index

In [ ]:
# Quick data inspection:
data.tail()

### **Step 2: Moving Averages Calculation**

Calculate short-term and long-term moving averages (e.g., 50-day and 200-day moving averages).

In [ ]:
short_window = 50
long_window = 200

data['Short_MA'] = data['Close'].rolling(window=short_window).mean()
data['Long_MA'] = data['Close'].rolling(window=long_window).mean()

### **Step 3: Generate Signals**

Create buy and sell signals based on the moving averages. For example, buy when the short-term moving average crosses above the long-term moving average (Golden Cross) and sell when it crosses below (Death Cross).

In [ ]:
# Step 3: Generate Signals
data['Signal'] = 0
data.loc[:, 'Signal'] = np.where(data['Short_MA'] > data['Long_MA'], 1, 0)
data['Position'] = data['Signal'].diff()

### **Step 4: Portfolio Simulation**

Simulate a portfolio starting with $10,000.
Track the value of the portfolio over time based on the signals generated.

In [ ]:
initial_capital = 10000
data['Portfolio_Value'] = initial_capital + (data['Position'].cumsum() * data['Close'].diff()).fillna(0).cumsum()

### **Step 5: Backtesting**

Evaluate the performance of the strategy, including metrics like total return, annualized return, volatility, maximum drawdown, and Sharpe ratio.

In [ ]:
total_return = data['Portfolio_Value'].iloc[-1] - initial_capital
annualized_return = (total_return / initial_capital) / ((data.index[-1] - data.index[0]).days / 365)
volatility = data['Portfolio_Value'].pct_change().std() * np.sqrt(252)  # Annualized volatility
max_drawdown = (data['Portfolio_Value'] / data['Portfolio_Value'].cummax()).min() - 1

### **Step 6: Visualization**

Plot the price of the index, moving averages, and buy/sell signals on a chart.

In [ ]:
# Visualization
plt.figure(figsize=(14, 7))
plt.plot(data['Date'], data['Close'], label='S&P 500 Price', alpha=0.5)
plt.plot(data['Date'], data['Short_MA'], label='50-Day MA', alpha=0.75)
plt.plot(data['Date'], data['Long_MA'], label='200-Day MA', alpha=0.75)

# Buy signals
plt.plot(data[data['Position'] == 1]['Date'],
         data['Short_MA'][data['Position'] == 1],
         '^', markersize=10, color='g', lw=0, label='Buy Signal')

# Sell signals
plt.plot(data[data['Position'] == -1]['Date'],
         data['Short_MA'][data['Position'] == -1],
         'v', markersize=10, color='r', lw=0, label='Sell Signal')

plt.title('S&P 500 Momentum Strategy with Moving Averages')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.grid()
plt.show()

### **Step 7: Summary Report**

Summarize the findings, including the performance metrics and any insights gained from the backtest.

In [ ]:
# Step 7: Summary Report
print(f'Total Return: ${total_return:.2f}')
print(f'Annualized Return: {annualized_return:.2%}')
print(f'Volatility: {volatility:.2%}')
print(f'Maximum Drawdown: {max_drawdown:.2%}')